<a href="https://www.kaggle.com/code/ibkya12/map-0-97-leashmp-competition?scriptVersionId=180413127" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 41.2 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 56.2 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 200000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 200000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,53476684,Cc1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c1C,CCC(CC)(CN)OC,Cn1cncc1C(CN)N1CCCC1,CCC(CC)(CNc1nc(NCC(c2cncn2C)N2CCCC2)nc(Nc2c(C(...,HSA,0
1,190650104,O=C(Nc1ccc(C(=O)O)cc1Cl)OCC1c2ccccc2-c2ccccc21,Nc1nccs1,COc1ccccc1-c1nnc(N)s1,COc1ccccc1-c1nnc(Nc2nc(Nc3nccs3)nc(Nc3ccc(C(=O...,sEH,0
2,139466377,O=C(Nc1c(C(=O)O)cccc1C(=O)O)OCC1c2ccccc2-c2ccc...,C=CCSCCN,Cl.NCC1CC(CC(N)=O)CO1,C=CCSCCNc1nc(NCC2CC(CC(N)=O)CO2)nc(Nc2c(C(=O)O...,HSA,0
3,126403164,O=C(N[C@H](Cc1ccccc1Cl)C(=O)O)OCC1c2ccccc2-c2c...,CC(C)(C)OC(=O)N1CCN(c2ccccc2N)CC1,C=CCNC(=O)CN.Cl,C=CCNC(=O)CNc1nc(Nc2ccccc2N2CCN(C(=O)OC(C)(C)C...,BRD4,0
4,180271706,O=C(Nc1ccc(Br)c(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1cncc(F)c1,Cc1ccnc(N)c1,Cc1ccnc(Nc2nc(NCc3cncc(F)c3)nc(Nc3ccc(Br)c(C(=...,sEH,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
from xgboost import XGBClassifier

In [8]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


In [9]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.120958
0:	learn: 0.6521420	total: 208ms	remaining: 3m 27s
1:	learn: 0.6168867	total: 372ms	remaining: 3m 5s
2:	learn: 0.5934204	total: 509ms	remaining: 2m 49s
3:	learn: 0.5744652	total: 675ms	remaining: 2m 48s
4:	learn: 0.5594357	total: 804ms	remaining: 2m 40s
5:	learn: 0.5451674	total: 956ms	remaining: 2m 38s
6:	learn: 0.5348190	total: 1.08s	remaining: 2m 33s
7:	learn: 0.5231440	total: 1.22s	remaining: 2m 31s
8:	learn: 0.5150600	total: 1.34s	remaining: 2m 28s
9:	learn: 0.5062544	total: 1.49s	remaining: 2m 27s
10:	learn: 0.5011246	total: 1.61s	remaining: 2m 24s
11:	learn: 0.4950908	total: 1.75s	remaining: 2m 24s
12:	learn: 0.4903281	total: 1.88s	remaining: 2m 22s
13:	learn: 0.4849509	total: 2s	remaining: 2m 20s
14:	learn: 0.4793094	total: 2.13s	remaining: 2m 20s
15:	learn: 0.4744772	total: 2.28s	remaining: 2m 20s
16:	learn: 0.4690475	total: 2.41s	remaining: 2m 19s
17:	learn: 0.4639694	total: 2.55s	remaining: 2m 18s
18:	learn: 0.4582003	total: 2.7s	remaining: 2m 1

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [10]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
